In [1]:
import cv2
import numpy as np
import utils
import inspect

In [2]:
width = 700
height = 700
questions = 5
choices = 5
ans = [0,1,1,2,4]

In [3]:
path = "./images/a.jpg"

In [4]:
webcamFeed = True
cameraNo = 0

cap = cv2.VideoCapture(cameraNo)
cap.set(10, 150)
while True: 
    if webcamFeed:success, img = cap.read()
    else: 
        img = cv2.imread(path)

    img_res = cv2.resize(img, (width, height))
    imgFinal = img_res.copy()
    imgFinal2 = img_res.copy()
    imgGray = cv2.cvtColor(img_res, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (5,5), 1)
    # Sử dụng hàm để phát hiện biên
    mag, angle = utils.sobel(imgBlur)
    non_max = utils.nonMax(mag, angle)
    imgCanny = utils.hysteresis(non_max)
    #Sử dụng thư viện
    #imgCanny = cv2.Canny(imgBlur, 10, 50)
    imgContours = img_res.copy() 
    imgContours2 = img_res.copy()
    imgBiggestContours = img_res.copy()

    try:
        contours, hierarchy = cv2.findContours(imgCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cv2.drawContours(imgContours, contours, -1, (0, 255, 0), 10)
        rectCon, unRect =  utils.rectContour(contours)
        cv2.drawContours(imgContours2, unRect, -1, (0, 255, 0), 10)
        biggestContours = utils.getCornerPoints(rectCon[0])
        gradePoints = utils.getCornerPoints(rectCon[1])

        if biggestContours.size != 0 and gradePoints.size != 0:
            biggestPoints=utils.reorder(biggestContours) 
            cv2.drawContours(imgBiggestContours, biggestPoints, -1, (0, 255, 0), 20)
            pt1 = np.float32(biggestPoints) 
            pt2 = np.float32([[0, 0],[width, 0], [0, height],[width, height]]) 
            matrix = cv2.getPerspectiveTransform(pt1, pt2)
            imgWarpColored = cv2.warpPerspective(img_res, matrix, (width, height))

            cv2.drawContours(imgBiggestContours, gradePoints, -1, (255, 0, 0), 20)
            gradePoints = utils.reorder(gradePoints) 
            ptG1 = np.float32(gradePoints) 
            ptG2 = np.float32([[0, 0], [325, 0], [0, 150], [325, 150]]) 
            
            matrixG = cv2.getPerspectiveTransform(ptG1, ptG2)
            imgGradeDisplay = cv2.warpPerspective(img_res, matrixG, (325, 150))
            imgWarpGray = cv2.cvtColor(imgWarpColored, cv2.COLOR_BGR2GRAY)
            imgThresh = cv2.threshold(imgWarpGray, 170, 255, cv2.THRESH_BINARY_INV)[1]

            boxes = utils.splitBoxes(imgThresh)
            myPixelVal = np.zeros((questions, choices))
            countC = 0
            countR = 0
            for image in boxes: 
                totalPixels = cv2.countNonZero(image)
                myPixelVal[countR][countC] = totalPixels
                countC+=1 
                if(countC == choices):
                    countR += 1
                    countC = 0
            print(myPixelVal)     

            myIndex = []
            unmarkedQuestions = []
            myIndex2 = []
            threshold = 4500
            threshold2 = 2000
            for x in range(0, questions):
                arr = myPixelVal[x]
                myIndexVal = np.where(arr > threshold) 
                myIndexVal2 = np.where(arr <= threshold) 
                if len(myIndexVal[0]) != 1:  
                    myIndex.append(-1) 
                else:  
                    myIndex.append(myIndexVal[0][0])   
                if len(myIndexVal2[0]) == 0:
                    myIndexVal2 = (np.array(range(choices)),)    
                for index in myIndexVal2[0]:  
                    unmarkedQuestions.append((x, index))  
            print("Unmarked Questions:", unmarkedQuestions)          
            print(myIndex)

            for x in range(0, questions):
                arr = myPixelVal[x]
                myIndexVal = np.where(arr > threshold2) 
                print(myIndexVal[0])    
                if len(myIndexVal[0]) != 1:  
                    myIndex2.append(-1)  
                else:  
                    myIndex2.append(myIndexVal[0][0])  
            print(myIndex2)

            grading = []
            grading2 = []
            for x in range(0, questions): 
                if ans[x] == myIndex[x]: 
                    grading.append(1)
                else: 
                    grading.append(0)
            score = (sum(grading)/questions)*100

            for x in range(0, questions): 
                if ans[x] == myIndex2[x]: 
                    grading2.append(1)
                else: 
                    grading2.append(0)
            score2 = (sum(grading2)/questions)*100

            #Thresh1
            imgResult = imgWarpColored.copy()
            imgResult = utils.showAnswers(imgResult, myIndex, grading, ans, questions,choices)
            imgRawDrawing = np.zeros_like(imgWarpColored)
            imgRawDrawing = utils.showAnswers(imgRawDrawing, myIndex, grading, ans, questions,choices)
            invMatrix = cv2.getPerspectiveTransform(pt2, pt1)
            imgInvWarp = cv2.warpPerspective(imgRawDrawing, invMatrix, (width, height))

            imgRawGrade = np.zeros_like(imgGradeDisplay)
            cv2.putText(imgRawGrade, str(int(score)) + "%", (50, 100), cv2.FONT_HERSHEY_COMPLEX, 3, (0, 255, 255), 3)
            invMatrixG = cv2.getPerspectiveTransform(ptG2, ptG1)
            imgInvGradeDisplay = cv2.warpPerspective(imgRawGrade, invMatrixG, (width, height))

            #Thresh2
            imgResult2 = imgWarpColored.copy()
            imgResult2 = utils.showAnswers(imgResult2, myIndex2, grading2, ans, questions,choices)
            imgRawDrawing2 = np.zeros_like(imgWarpColored)
            imgRawDrawing2 = utils.showAnswers(imgRawDrawing2, myIndex2, grading2, ans, questions,choices)
            invMatrix2 = cv2.getPerspectiveTransform(pt2, pt1)
            imgInvWarp2 = cv2.warpPerspective(imgRawDrawing2, invMatrix2, (width, height))

            imgRawGrade2 = np.zeros_like(imgGradeDisplay)
            cv2.putText(imgRawGrade2, str(int(score2)) + "%", (50, 100), cv2.FONT_HERSHEY_COMPLEX, 3, (0, 255, 255), 3)
            invMatrixG2 = cv2.getPerspectiveTransform(ptG2, ptG1)
            imgInvGradeDisplay2 = cv2.warpPerspective(imgRawGrade2, invMatrixG2, (width, height))
            #show unmark
            imgUnmark = imgWarpColored.copy()
            imgUnmark = utils.showUnMark(imgUnmark, myIndex,  unmarkedQuestions, grading, ans, questions,choices)
            #Final thresh1
            imgFinal = cv2.addWeighted(imgFinal, 1, imgInvWarp, 1, 0)
            imgFinal = cv2.addWeighted(imgFinal, 0.90, imgInvGradeDisplay, 1, 0)
            #Final thresh2
            imgFinal2 = cv2.addWeighted(imgFinal2, 1, imgInvWarp2, 1, 0)
            imgFinal2 = cv2.addWeighted(imgFinal2, 0.90, imgInvGradeDisplay2, 1, 0)
        imgBlank = np.zeros_like(img_res)
        imageContoursArray = ([img_res, imgGray, imgBlur, imgCanny],[imgContours, imgBiggestContours, imgWarpColored, imgThresh],[imgResult, imgRawDrawing,imgInvWarp, imgFinal]) 
        imageContoursArray2 = ([imgContours, imgContours2, imgUnmark, imgBlank])
        imageContoursArray3 = ([img_res, imgGray, imgBlur, imgCanny],[imgContours, imgBiggestContours, imgWarpColored, imgThresh],[imgResult2, imgRawDrawing2,imgInvWarp2, imgFinal2])
    except: 
        imgBlank = np.zeros_like(img_res)
        imageContoursArray = ([img_res, imgGray, imgBlur, imgCanny],[imgBlank, imgBlank, imgBlank, imgBlank],[imgBlank, imgBlank,imgBlank, imgBlank])     
        imageContoursArray3 = ([img_res, imgGray, imgBlur, imgCanny],[imgBlank, imgBlank, imgBlank, imgBlank],[imgBlank, imgBlank,imgBlank, imgBlank]) 
        imageContoursArray2 = ([imgContours, imgContours2, imgUnmark, imgBlank])
    #lables = [["Original", "Gray", "Blur", "Canny"],["Contours", "Biggest Con", "Warp", "Threshold"], ["Result", "Raw Drawing", "Inv Warp", "Final"]]
    imgContoursStacked = utils.stackImages(imageContoursArray, 0.3)
    imgContoursStacked2 = utils.stackImages(imageContoursArray2, 0.3)
    imageContoursStack3 = utils.stackImages(imageContoursArray3, 0.3)
    #cv2.imshow("Final Result", imgFinal)
    cv2.imshow('Thresh 1', imgContoursStacked)
    #cv2.imshow('Request', imgContoursStacked2)
    #cv2.imshow('Thresh 2', imageContoursStack3)
    if cv2.waitKey(1) & 0xFF == ord('s'):
        cv2.imwrite("FinalResult.jpg", imgFinal)
        cv2.waitKey(300)

NameError: name 'imgUnmark' is not defined